# MuseNet 1B4 (Basemodel Training)
This model is a custom 1.4B model containing
- 96 layers
- 1024 embedding size

It was initialized using the original RWKV trainer here : https://github.com/PicoCreator/RWKV-LM-LoRA/blob/picocreator-init-memory-experiment/notebook/echo-B-1B4-training.ipynb

This is just a silly experiment of using RWKV with music

> This project assumes you have the rwkv-infctx conda env setup, and you are executing in that environment - see the main README.md for the conda env setup steps

# Basic Setup

In [ ]:
# First lets get the blank init model, these init model was generated
# using the original RWKV-LM repo (as at this point of writing, this repo cannot init a model)
#
# As such I have preinitialized these blank models and uploaded them to HF for convinence
!mkdir -p ../../../model/
!mkdir -p ../../../datapath/
!mkdir -p ../../../checkpoint/
!cd ../../../model/ && wget https://huggingface.co/picocreator/memory-size-experiment-for-rwkv/resolve/main/Echo-B-1B4-Init.pth

In [ ]:
DEEPSPEED_STRAT="deepspeed_stage_2_offload"
GPU_DEVICES="auto"
ENABLE_WANDB=False
WANDB_PREFIX="Musenet-1B4 L96-D1024"

print("DEEPSPEED_STRAT:", DEEPSPEED_STRAT)
print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v4neo/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

## Dataset preloading

In [ ]:
# Lets preload the requried dataset (enwiki_100k)
!cd "{TRAINER_DIR}" && \
    python3 preload_dataset.py "{NOTEBOOK_DIR}/Musenet-1B4.yaml"

## Training process

In [ ]:
# Foundation model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python new_train.py fit \
        -c "{NOTEBOOK_DIR}/Musenet-1B4.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - (train-ctx=4096, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" 

In [ ]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py "../checkpoint/Musenet-1B4/last.ckpt" "../model/Musenet-1B4.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/Musenet-1B4.pth"